In [18]:
import pandas as pd
df=pd.read_csv("smsa_cleaned.csv")


In [19]:
import re

def clean_text(text):
    text = text.strip()
    text = re.sub(r"http\S+", "[URL]", text)        # ganti url
    text = re.sub(r"@\w+", "[USER]", text)          # ganti mention
    text = re.sub(r"\d+", "[NUM]", text)            # ganti angka
    return text

# Cleaning
df['sentence'] = df['sentence'].apply(clean_text)

# Mapping label angka → kategori
label_mapping = {"negative":0,"neutral":1, "positive":2}
df['label_angka'] = df['label'].map(label_mapping)

# Simpan ke CSV
df.to_csv("smsa_cleaned.csv", index=False, encoding="utf-8")
print("✅ Dataset berhasil disimpan ke smsa_cleaned.csv")


✅ Dataset berhasil disimpan ke smsa_cleaned.csv


In [20]:
# Menampilkan contoh komentar spam (label = 1)
print("=== Contoh Komentar Negatif (Label = 0) ===")
spam_samples = df[df['label_angka'] == 0]['sentence'].head(10)
for i, comment in enumerate(spam_samples, 1):
    print(f"{i}. {comment}")
    print()

=== Contoh Komentar Negatif (Label = 0) ===
1. duh . jadi mahasiswa jangan sombong dong . kasih kartu kuning segala . belajar dulu yang baik , tidak usahlah ikut-ikut politik . nanti sudah selesai kuliah nya mau ikut politik juga tidak telat . dasar mahasiswa .

2. pakai kartu kredit bca tidak untung malah rugi besar

3. simcard indosat inaktiv gara-gara lupa isi pulsa dan kabar nya aktif jika pinda ke pasca bayar , ribet banget

4. sifat iri sering muncul pada orang orang yang tidak punya tujuan hidup

5. pengalaman bersama indosat hari ini , semoga tidak terjadi pada pelanggan lain . sempat sampai marah-marah dengan pelayanan pelanggan indosat

6. anak sekarang sulit untuk dinasehati

7. hanya mengatasi masalah kayak gini saja anies sandi tidak becus . ke mana saja pasukan kebersihan yang begitu banyak jumlah nya ?

8. alhamdulillah hari ini tidak ke jalan bugel agak kesal juga kalau tiap hari lewat sana melulu . sempit , dua arah , macet sudah jadi makanan sehari-hari

9. respon aga

In [21]:
# Menampilkan contoh komentar spam (label = 1)
print("=== Contoh Komentar Netral (Label = 1) ===")
spam_samples = df[df['label_angka'] == 1]['sentence'].head(10)
for i, comment in enumerate(spam_samples, 1):
    print(f"{i}. {comment}")
    print()

=== Contoh Komentar Netral (Label = 1) ===
1. mohon ulama lurus dan k[NUM] mmbri hujjah partai apa yang harus diwlh agar suara islam tidak pecah-pecah

2. setiap bumn dibentuk dengan uu bukan dibentuk dengan uu .

3. pdip sebut ridwan kamil menang karena berbaju merah

4. [NUM]:[NUM] : kedatangan presiden ri dan rombongan di gate [NUM] , terminal [NUM] disambut oleh menteri perhubungan , menteri bumn , gubernur banten , dirjen perkeretaapian kemhub , dirut ap[NUM] , dirut pt kami , dan dirut railink

5. risma menjatuhkan pilihan nya pada gus ipul - puti di pilgub jatim [NUM] ini .

6. hai tolong bantu saya tidak bisa input tidak resi jne dan kenomor resi sudah dipakai .

7. mardani ali sera : kemenangan di sosial media , akan selalu kemenangan di dunia nyata .

8. singapura selidiki penjualan prt indonesia di situs daring

9. suasana di kantor dpc kab. bogor menjelang pidato awal tahun [NUM] oleh

10. hasil quick count pilgub kaltim [NUM] , isran - hadi unggul



In [22]:
# Menampilkan contoh komentar spam (label = 1)
print("=== Contoh Komentar Positif (Label = 2) ===")
spam_samples = df[df['label_angka'] == 2]['sentence'].head(10)
for i, comment in enumerate(spam_samples, 1):
    print(f"{i}. {comment}")
    print()

=== Contoh Komentar Positif (Label = 2) ===
1. warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !

2. lokasi strategis di jalan sumatera bandung . tempat nya nyaman terutama sofa di lantai [NUM] . paella nya enak , sangat pas dimakan dengan minum bir dingin . appetiser nya juga enak-enak .

3. betapa bahagia nya diri ini saat unboxing paket dan barang nya bagus ! menetapkan beli lagi !

4. makanan beragam , harga makanan di food stall akan ditambahkan [NUM] % lagi di kasir , suasana ramai dan perlu perhatian untuk mendapatkan parkir dan tempat duduk .

5. tempat unik , bagus buat foto , makanan enak , pegawai ramah , bersih dan luas , wif

In [23]:
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")

# Split data train-test
X_train, X_test, y_train, y_test = train_test_split(
    df['sentence'], df['label_angka'], test_size=0.2, stratify=df['label_angka'], random_state=42
)

def tokenize(batch):
    return tokenizer(batch["sentence"], padding="max_length", truncation=True, max_length=128)

from datasets import Dataset
train_dataset = Dataset.from_dict({"sentence": X_train, "label_angka": y_train})
test_dataset = Dataset.from_dict({"sentence": X_test, "label_angka": y_test})

train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Saat bikin dataset
train_dataset = train_dataset.rename_column("label_angka", "labels")
test_dataset = test_dataset.rename_column("label_angka", "labels")

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/8800 [00:00<?, ? examples/s]

Map:   0%|          | 0/2200 [00:00<?, ? examples/s]

In [24]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained(
    "indobenchmark/indobert-base-p1",
    num_labels=3
)

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=20,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dewandaca18 (dewandacaa18) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.209600,0.190134,0.933182,0.933376,0.933182,0.932974
2,0.103200,0.212686,0.943636,0.943404,0.943636,0.943463
3,0.053200,0.250352,0.942727,0.942541,0.942727,0.942516


TrainOutput(global_step=1650, training_loss=0.13954594507361903, metrics={'train_runtime': 841.1227, 'train_samples_per_second': 31.387, 'train_steps_per_second': 1.962, 'total_flos': 1736548557004800.0, 'train_loss': 0.13954594507361903, 'epoch': 3.0})

In [25]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

predictions = trainer.predict(test_dataset)
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

print(classification_report(y_true, y_pred, target_names=["negative","neutral","positif"]))
print(confusion_matrix(y_true, y_pred))


              precision    recall  f1-score   support

    negative       0.93      0.92      0.93       687
     neutral       0.91      0.88      0.90       230
     positif       0.96      0.97      0.96      1283

    accuracy                           0.94      2200
   macro avg       0.93      0.92      0.93      2200
weighted avg       0.94      0.94      0.94      2200

[[ 634   11   42]
 [  13  202   15]
 [  35    8 1240]]


In [26]:
# 0. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 7. Save Model & Tokenizer ke Google Drive
save_dir = "/content/drive/MyDrive/indobert-sentiment"  # folder tujuan di GDrive
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

Mounted at /content/drive


('/content/drive/MyDrive/indobert-sentiment/tokenizer_config.json',
 '/content/drive/MyDrive/indobert-sentiment/special_tokens_map.json',
 '/content/drive/MyDrive/indobert-sentiment/vocab.txt',
 '/content/drive/MyDrive/indobert-sentiment/added_tokens.json')

In [30]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

model = BertForSequenceClassification.from_pretrained(save_dir)
tokenizer = BertTokenizer.from_pretrained(save_dir)

# Fungsi Prediksi
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = outputs.logits.argmax(-1).item()
    return ["negative","neutral","positif"][prediction]

# Contoh Deployment
print(predict_sentiment("Film ini sangat bagus sekali"))
print(predict_sentiment("Aplikasi ini bikin kesal, sering error"))
print(predict_sentiment("Aplikasi tidak bermanfaat"))

positif
negative
negative
